In [2]:
import numpy as np 
import sqlalchemy as sa
import datetime as datetime
from datetime import timedelta
import pandas as pd
import pymysql
import scipy.stats as norm
import inventorize as inv

In [3]:
con=sa.create_engine('mysql+pymysql://{user}:{pw}@{host}/{db}'.format
                     (user='root',pw='ghar123+bi',host='localhost:3306',db='mango'))

stocks_three_month= pd.read_sql("""SELECT * from mango.mango_stocks WHERE DATE(Date)<= 
(select max(DATE(Date)) from mango.mango_stocks) AND 
DATE(Date)>= (select max(DATE(Date))- interval 90 DAY from mango.mango_stocks);""",con)


sales_two_years= pd.read_sql("""SELECT * from mango.mango_sales WHERE DATE(Date)<= 
(select max(DATE(Date)) from mango.mango_sales) AND 
DATE(Date)>= (select max(DATE(Date))- interval 730 DAY from mango.mango_sales);""",con)

orders= pd.read_sql("""SELECT * from mango.mango_orders WHERE DATE(Date)<= 
(select max(DATE(Date)) from mango.mango_orders) AND 
DATE(Date)>= (select max(DATE(Date))- interval 10 DAY from mango.mango_orders);""",con)


sales_three_month=  pd.read_sql("""SELECT * from mango.mango_sales WHERE DATE(Date)<= 
(select max(DATE(Date)) from mango.mango_sales) AND 
DATE(Date)>= (select max(DATE(Date))- interval 90 DAY from mango.mango_sales);""",con)


In [4]:
stocks_three_month['Date']=pd.to_datetime(stocks_three_month['Date'])
sales_three_month['Date']=pd.to_datetime(sales_three_month['Date'])
orders['Date']=pd.to_datetime(orders['Date'])
sales_two_years['Date']=pd.to_datetime(sales_two_years['Date'])

In [5]:
sales_three_month['label']=sales_three_month['description']+'_'+sales_three_month['size']+'_'+sales_three_month['color']+'_'+sales_three_month['section']+'_'+sales_three_month['subfamily']+'_'+sales_three_month['brand']

In [6]:
orders['label']=(orders['description']+'_'+orders['size']+'_'+orders['color']+'_'+orders['section']+'_'+orders['subfamily']+'_'+orders['brand'])

In [7]:
sales_two_years['month']=sales_two_years['Date'].dt.month
sales_two_years['year']=sales_two_years['Date'].dt.year

In [8]:
sales_two_years['label']=(sales_two_years['description']+'_'+sales_two_years['size']+'_'+sales_two_years['color']+'_'+sales_two_years['section']+'_'+sales_two_years['subfamily']+'_'+sales_two_years['brand'])

In [9]:
sales_three_month=sales_three_month.groupby(['label','Date']).agg(sales=('Qty',np.sum)).reset_index()

In [10]:

average_category=(sales_two_years.groupby(['month','section','subfamily']).agg(sales=('Qty',np.sum)).reset_index().groupby(['section','subfamily']).agg(average_category=('sales',np.mean)).reset_index())

In [11]:
average_monthly=(sales_two_years.groupby(['year','month','section','subfamily']).agg(sales=('Qty',np.sum)).reset_index().groupby(['month','section','subfamily']).agg(average_monthly=('sales',np.mean
)).reset_index())

In [12]:
seasonality=pd.merge(average_monthly,average_category,how='outer')

In [13]:
seasonality['seasonality']=seasonality['average_monthly']/seasonality['average_category']

In [14]:

seasonality=seasonality.loc[seasonality.month==datetime.datetime.now().month,:].reset_index()

In [15]:
seasonality.drop('index',1,inplace=True)
seasonality

C:\Users\Brahim\AppData\Local\Temp\ipykernel_11584\1613469921.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  seasonality.drop('index',1,inplace=True)


,month,section,subfamily,average_monthly,average_category,seasonality
0,2,cycling,jackets,8.0,9.916667,0.806723
1,2,cycling,shorts,54.5,96.666667,0.563793
2,2,fitness,pants,5.0,24.833333,0.201342
3,2,fitness,shorts,59.0,135.916667,0.434090
4,2,fitness,tops,41.0,65.000000,0.630769
5,2,outdoors,jackets,295.5,503.416667,0.586989
6,2,outdoors,pants,255.5,477.666667,0.534892
7,2,outdoors,shorts,26.0,78.500000,0.331210
8,2,outdoors,underwear,23.5,49.000000,0.479592
9,2,outing,Pefumes,1646.5,3592.000000,0.458380


In [16]:
stocks_three_month.head(1)

,index,Date,Inventory,Description,size,color,section,subfamily,brand
0,90,2019-10-27,0.0,item 1000,13,White,swimming,shorts,Delphine Kemmer


In [17]:
stocks_three_month['label']=(stocks_three_month['Description']+'_'+stocks_three_month['size']+'_'+stocks_three_month['color']+'_'+stocks_three_month['section']+'_'+stocks_three_month['subfamily']+'_'+stocks_three_month['brand'])

In [18]:
stocks_three_month.head(1)

,index,Date,Inventory,Description,size,color,section,subfamily,brand,label
0,90,2019-10-27,0.0,item 1000,13,White,swimming,shorts,Delphine Kemmer,item 1000_13_White_swimming_shorts_Delphine K...


In [19]:
stocks_three_month['begining']=stocks_three_month['Inventory']
stocks_three_month.drop('Inventory',axis=1,inplace=True)

In [21]:
stocks_three_month['ending']=stocks_three_month.groupby('label')['begining'].transform(lambda x: x.shift(-1))


In [22]:
dataset=stocks_three_month.copy()

In [23]:
dataset.head(1)

,index,Date,Description,size,color,section,subfamily,brand,label,begining,ending
0,90,2019-10-27,item 1000,13,White,swimming,shorts,Delphine Kemmer,item 1000_13_White_swimming_shorts_Delphine K...,0.0,0.0


In [24]:
dataset=pd.merge(dataset,sales_three_month,how='outer',on=['Date','label'])

In [25]:
dataset.fillna(0,inplace=True)
dataset['Received']=dataset['ending']+dataset['sales']-dataset['begining']

In [22]:
def stock_status(data):
    if((data.begining<=0)&(data.sales==0)&(data.Received<=0)):
        a='out_of_stocks'
    elif ((data.begining<=0)&(data.sales==0)):
        a='out_of_stocks'
    elif ((data.begining>0)&(data.sales==0)):
        a='no_sales'
    elif ((data.sales>0)):
        a='selling_day'
    else:
        a='question_mark'
    return a

In [23]:
dataset['status']=dataset.apply(stock_status,axis=1)

In [24]:
sales_two_years['revenue']=sales_two_years['price_paid']/sales_two_years['Qty']
sales_two_years['profit']=(sales_two_years['price_paid' ]/sales_two_years['Qty'])-sales_two_years['Cost']

In [25]:
product_attribute=sales_two_years.groupby('label').agg(Cost=('Cost',np.mean),revenue=('revenue',np.mean),profit=('profit',np.mean))

In [26]:
dataset=pd.merge(dataset,product_attribute,how='left',on='label')

In [27]:
max_date_requested=max(orders['Date'])
min_date_requested=max_date_requested-timedelta(days=10)
leadtime=10
total_requested=(orders[(orders['Date']<=max_date_requested)&(orders['Date']>=min_date_requested)].groupby('label').agg(total_requested=('Qty',np.sum)).reset_index)

In [28]:
curent_stock=dataset[dataset['Date']==max(dataset['Date'])].groupby('label').agg(beginning=('begining',np.mean))

In [29]:
for_abc=dataset.groupby('label').agg(sales=('sales',np.sum),revenue=('revenue',np.mean)).sort_values(by='sales',ascending=False).reset_index()

In [30]:
for_abc.head(1)

,label,sales,revenue
0,item 3902_Standard_Standard_outing_Pefumes_Ne...,298.0,287.063079


In [31]:
for_abc.fillna(0,inplace=True)

In [32]:
abc=inv.productmix(for_abc['label'],for_abc['sales'],for_abc['revenue'])

In [33]:
mapping_driver={
'A_A':'Volume and margin driver',
'A_B':'Volume driver',
'A_C':'Volume driver',
'C_A':'Margin driver',
'B_A':'Margin driver',
'B_C':'Regulars',
'C_B':'Regulars',
'B_B':'Regulars',
'C_C':'Slow moving'
}

In [34]:
abc['drivers']=abc['product_mix'].map(mapping_driver)

In [35]:
abc.rename(columns={'skus':'label'},inplace=True)

In [36]:
inventory_cal=dataset.groupby('label').agg(demande_avg=('sales',lambda x: x[dataset['status'] != 'out_of_stock'].mean()),
sd=('sales','sd'),stock_cost=('Cost', lambda x: x[dataset['Cost']>0].mean()),
days_of_sales=('sales', lambda x: x[dataset['sales']>0].count()),
days_of_stocks=('begining', lambda x: x[dataset['begining']>0].count()))

In [ ]:
inventory_cal.reset_index(inplace=True)

In [ ]:
inventory_cal['selling_rate_days']=inventory_cal['days_of_sales']/inventory_cal['days_of_stocks']

In [ ]:
inventory_cal=pd.mercge(inventory_cal,abc[['label','sales','revenue','product_mix','deivers']],how='left')